In [1]:
import pandas as pd
import numpy as np
import gc
import datetime
import time

%matplotlib inline

In [2]:
st = time.time()

adTrain = pd.read_csv('c:\\kaggle\\adtracking\\train.csv', skiprows=range(1,150000000))

adTrain.info()
adTrain.head()

print(time.time()- st)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34903891 entries, 0 to 34903890
Data columns (total 8 columns):
ip                 int64
app                int64
device             int64
os                 int64
channel            int64
click_time         object
attributed_time    object
is_attributed      int64
dtypes: int64(6), object(2)
memory usage: 2.1+ GB
94.72941827774048


In [3]:
'''
adAttributed = pd.read_csv('c:\\kaggle\\adtracking\\attributed.csv', skiprows=0)

frames = [adTrain, adAttributed]
adTrain = pd.concat(frames)

del adAttributed
gc.collect()
'''

"\nadAttributed = pd.read_csv('c:\\kaggle\\adtracking\\attributed.csv', skiprows=0)\n\nframes = [adTrain, adAttributed]\nadTrain = pd.concat(frames)\n\ndel adAttributed\ngc.collect()\n"

In [4]:
st = time.time()

adTrain['click_time'] = pd.to_datetime(adTrain['click_time'])
adTrain['click_hour'] = adTrain['click_time'].dt.hour
adTrain.drop(['click_time','attributed_time'], axis=1, inplace=True)
gc.collect()
print(time.time()- st)

13.146752119064331


In [5]:
import lightgbm as lgb

st = time.time()
X_train = adTrain[['ip', 'app', 'device', 'os', 'channel', 'click_hour']]
y_train = adTrain['is_attributed']

mdl = lgb.LGBMClassifier(n_estimators=200)
mdl.fit(X_train, y_train)

#del adTrain
#gc.collect()
print(mdl)
print(time.time()- st)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=200,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)
221.7806851863861


In [6]:
st = time.time()

adTest = pd.read_csv('c:\\kaggle\\adtracking\\train.csv', skiprows=range(1,50000000), nrows=10000000)

adTest['click_time'] = pd.to_datetime(adTest['click_time'])
adTest['click_hour'] = adTest['click_time'].dt.hour
adTest.drop(['click_time','attributed_time'], axis=1, inplace=True)
gc.collect()

print(time.time()- st)

28.889652490615845


In [7]:
import sklearn.metrics as metrics

st = time.time()

X_test = adTest[['ip', 'app', 'device', 'os', 'channel', 'click_hour']]
y_test = adTest['is_attributed']

X_test_predict = mdl.predict_proba(X_test)
print(X_test_predict)
print(time.time()- st)

auc_roc = metrics.roc_auc_score(y_test, X_test_predict[::, 1])

# with attributed 0.9540315356090971
# default: 0.9698160235023366
# n_estimators=200: 0.9699698598812887
# n_estimators=200, learning_rate=0.05: 0.970567098425588
# boosting_type='dart': 0.9275187838276894
# boosting_type='goss': 0.9698931175924093
print(auc_roc)

[[9.99487889e-01 5.12111195e-04]
 [9.99835845e-01 1.64155226e-04]
 [9.99898418e-01 1.01582295e-04]
 ...
 [9.99604499e-01 3.95501304e-04]
 [9.99868113e-01 1.31887297e-04]
 [9.99917633e-01 8.23666355e-05]]
25.56046223640442
0.9703329550784882


In [8]:
st = time.time()

del adTest
gc.collect()

adTest = pd.read_csv('c:\\kaggle\\adtracking\\test.csv')

adTest['click_time'] = pd.to_datetime(adTest['click_time'])
adTest['click_hour'] = adTest['click_time'].dt.hour
adTest.drop(['click_time'], axis=1, inplace=True)
gc.collect()

adTest.info()

print(time.time()- st)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18790469 entries, 0 to 18790468
Data columns (total 7 columns):
click_id      int64
ip            int64
app           int64
device        int64
os            int64
channel       int64
click_hour    int64
dtypes: int64(7)
memory usage: 1003.5 MB
21.489229202270508


In [9]:
st = time.time()

ad_test = adTest[['ip', 'app', 'device', 'os', 'channel', 'click_hour']]

ad_test_predict = mdl.predict_proba(ad_test)
print(ad_test_predict)
print(time.time()- st)

[[9.99553421e-01 4.46578707e-04]
 [9.99633104e-01 3.66895927e-04]
 [9.99767244e-01 2.32756415e-04]
 ...
 [9.99794779e-01 2.05220825e-04]
 [9.99735348e-01 2.64651840e-04]
 [9.99957529e-01 4.24705499e-05]]
50.05686283111572


In [10]:
submit_file = 'c:\\kaggle\\adtracking\\Submission_lgbm.csv'
print(submit_file)
submission = pd.DataFrame({
        "click_id": adTest["click_id"],
        "is_attributed": ad_test_predict[::, 1]
    })

submission.to_csv(submit_file, index=False)

c:\kaggle\adtracking\Submission_lgbm.csv
